In [27]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


In [28]:
!pip install torchmetrics

In [29]:
import zipfile
zip_ref = zipfile.ZipFile('dogs-vs-cats.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [30]:
import torch
from torch.utils.data import  DataLoader
from torchvision import datasets, transforms
from torch import nn
from torchmetrics.classification  import BinaryAccuracy

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [32]:
transform = transforms.Compose([
    transforms.Resize((288,228)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [55]:
train = datasets.ImageFolder("/content/train", transform=transform)
test = datasets.ImageFolder("/content/test", transform=transform)
print(train.class_to_idx)

{'cats': 0, 'dogs': 1}


In [34]:
torch.manual_seed(42)
data_loader = DataLoader(train, batch_size=32, shuffle=True)

In [35]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()

    self.feature = nn.Sequential(
      #1
      nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding='valid'),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),
      #2
      nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=3, padding='valid'),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),
      #3
      nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size=3, padding='valid'),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.classifier = nn.Sequential(
      nn.Flatten(),
      #4
      nn.Linear(in_features = 128*34*26 , out_features=128),
      nn.ReLU(),
      #5
      nn.Linear(in_features=128, out_features=64),
      nn.ReLU(),
      #6
      nn.Linear(in_features=64, out_features=1),
      nn.Sigmoid()
    )

  def forward(self, x):
    x = self.feature(x)
    x = self.classifier(x)
    return x

In [44]:
def train(epochs, train_loader, model, criteration, optimizer, accuracy):
  loss_list = []
  accuracy_list = []
  for epoch in range(epochs):
    loss_batch = 0
    acc_sum = 0
    num_batches = 0
    for X, y in train_loader:
      X = X.to(device)
      y = y.to(device)
      y = y.view(-1, 1).float()
      #forward pass
      y_pred = model(X)
      #loss calculation
      loss = criteration(y_pred, y)
      #reset optimizer
      optimizer.zero_grad()
      #backpropagations
      loss.backward()
      #upgrade grad
      optimizer.step()

      loss_batch += loss.item()

      acc_sum += accuracy(y_pred, y).item()
      num_batches += 1

    loss_list.append(loss_batch/len(train_loader))
    avg_acc = acc_sum / num_batches
    accuracy_list.append(avg_acc)

    print(f"Epoch: {epoch} Loss: {loss_batch/len(train_loader)} Accuracy: {avg_acc}")

  return loss, accuracy

In [43]:
model.eval()

CNN(
  (feature): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=113152, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=1, bias=True)
    (6): Sigmoid()
  )
)

In [46]:
model = CNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()
accuracy = BinaryAccuracy(threshold=0.5).to(device)
train(
    10,
    data_loader,
    model,
    nn.BCELoss(),
    optimizer,
    accuracy
    )

Epoch: 0 Loss: 0.6010701561927796 Accuracy: 0.6565
Epoch: 1 Loss: 0.48101109809875486 Accuracy: 0.76825
Epoch: 2 Loss: 0.38208312666416167 Accuracy: 0.83055
Epoch: 3 Loss: 0.2781326563835144 Accuracy: 0.8821
Epoch: 4 Loss: 0.15733678929805756 Accuracy: 0.93845
Epoch: 5 Loss: 0.07147173039410264 Accuracy: 0.97525
Epoch: 6 Loss: 0.05782257976308465 Accuracy: 0.981
Epoch: 7 Loss: 0.03274508445552783 Accuracy: 0.98785
Epoch: 8 Loss: 0.03188078716802411 Accuracy: 0.9893
Epoch: 9 Loss: 0.02833693324624328 Accuracy: 0.99045


(tensor(0.0360, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>),
 BinaryAccuracy())

In [52]:
data_loader_test = DataLoader(test, batch_size=32, shuffle=False)

total_loss = 0
total_correct = 0
total_samples = 0

with torch.no_grad():
    for X_valid, y_valid in data_loader_test:
        X_valid = X_valid.to(device)
        y_valid = y_valid.to(device)
        y_valid = y_valid.view(-1, 1).float()

        pred = model(X_valid)
        loss = criterion(pred, y_valid)

        preds = (pred >= 0.5).int()

        total_loss += loss.item() * len(X_valid)
        total_correct += (preds == y_valid).sum().item()
        total_samples += len(X_valid)

avg_loss = total_loss / total_samples
avg_accuracy = total_correct / total_samples

print("###############################")
print(f'Average Validation Loss: {avg_loss:.4f} | Average Accuracy: {avg_accuracy:.4f}')
print("###############################")

###############################
Average Validation Loss: 1.1613 | Average Accuracy: 0.7830
###############################


In [53]:
torch.save(model.state_dict(), 'dogs_vs_cats_model.pth')